## Spots that are always excluded:
##### Test spots
##### Spots marked as problematic
##### Suscpicious for analysis == 'Yes' spots
##### Fake Sale spots: This means that Reason for Cancelling Survey is any combination which contains 'Fake Sale'
##### Spots whose stop date was before 2018


## CAN CANCEL:
##### Spots which cancelled more than 2 months before their Available Cancellation Date are excluded
##### Spots whose Available Cancellation Date is 2 or more months after the date of analysis
##### Months during which spots couldn't have cancelled according to the above conditions

## wo CB sets:
##### Closed Business, Sold Business and Non-payment spots are excluded. This means Reason for Cancelling Survey is any combination which contains 'Closed Business', 'Closed/Sold Business', 'Sold Business', 'Sold/Closed Business' or 'Non-payment'
## canc conf event
##### Spots for which Date Cancellation Confirmed is more than 60 days after Date Cancellation Requested

#### ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [1]:
import pandas as pd
import numpy as np
import importlib
import get_stopped_doing_something_variables
import prepare_for_the_models
import fit_tv_cox_models
#import read_vars_for_separate_models_for_dependent_vars
import read_a_combination_of_variables
import yaml
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display, Markdown

In [2]:
# read the yaml file with a list of parameters needed for the report
with open(r'./parameters/stopped_doing_something_report_parameters.yaml') as file:
    parameters = yaml.load(file, Loader=yaml.FullLoader)

date_of_analysis = parameters['date_of_analysis']
date_dir = date_of_analysis.replace('-', '_')
### name of the data directory ###
churn_based_on_behaviour_dir = parameters['churn_based_on_behaviour_dir']
### penalizer value when fitting the models ###
penalizer = parameters['penalizer']
### model type ###
model_type = parameters['model_type']
### coefficient and p values when dropping unsignificant variables ###
coefficient_limit_for_numerical_vars = parameters['coefficient_limit_for_numerical_vars']
coefficient_limit_for_cat_vars = parameters['coefficient_limit_for_cat_vars']
p_limit = parameters['p_limit']
additional_higher_p_limit = parameters['additional_higher_p_limit']
additional_lower_p_limit = parameters['additional_lower_p_limit']

# read the yaml file with data set parameters #
# data_set_name = input('Data set name: ')
data_set_name = 'ALL_spots_with_CB_cancellation_requested'
with open(r'./parameters/data_sets.yaml') as file:
    data_sets_parameters = yaml.load(file, Loader=yaml.FullLoader)
    
spots_set = data_sets_parameters[data_set_name]['spots_set']
with_wo_CB = data_sets_parameters[data_set_name]['with_wo_CB']
event_date_full_name = data_sets_parameters[data_set_name]['event_date_type']

if with_wo_CB == 'with_CB':
    with_wo_CB_boolean = True
else:
    with_wo_CB_boolean = False

if event_date_full_name == 'cancellation_requested':
    event_date = 'canc_req'
elif event_date_full_name == 'cancellation_confirmed':
    event_date = 'canc_conf'

In [3]:
##### base columns ##### 
base_cols = ['spot_id',\
            'time',\
            'event']

In [4]:
##### Model 0: all variables - prepare data for the model ####
cols_to_use = read_a_combination_of_variables.\
main(model_number=0, dir_name='combinations_of_variables_that_are_not_dependent/')
#### get behavioural variables ####
(variables_to_use_for_the_model, did_something_before_vars, did_something_last_month_vars) = \
get_stopped_doing_something_variables.main(date_of_analysis=date_of_analysis, variables_to_use_for_the_model=cols_to_use)
cols = base_cols + variables_to_use_for_the_model + \
did_something_before_vars + did_something_last_month_vars


#### get data for the model ###
(data, base_df, df_timeline_all_vars) = \
prepare_for_the_models.get_data_for_the_MV_Cox_model(date_of_analysis=date_of_analysis, spots_set=spots_set, \
                            with_wo_CB=with_wo_CB, event_date=event_date, columns=cols, data_dir=churn_based_on_behaviour_dir,\
                            C = 100)
    
model_numbers = \
read_a_combination_of_variables.get_a_list_of_model_numbers(dir_name='combinations_of_variables_that_are_not_dependent/')

In [5]:
for model_number in model_numbers[1:]:
    cols_to_use = read_a_combination_of_variables.\
    main(model_number=model_number, dir_name='combinations_of_variables_that_are_not_dependent/')
    
    model_name = \
    read_a_combination_of_variables.get_model_names(model_number=model_number, \
                                                    dir_name='combinations_of_variables_that_are_not_dependent/')
    display(Markdown("# Model "+ str(model_number) + ": " + model_name))

    #### get behavioural variables ####
    (variables_to_use_for_the_model, did_something_before_vars, did_something_last_month_vars) = \
    get_stopped_doing_something_variables.main(date_of_analysis=date_of_analysis, variables_to_use_for_the_model=cols_to_use)
    cols = variables_to_use_for_the_model + \
    did_something_before_vars + did_something_last_month_vars

    #### data for the model ###
    df_timeline = df_timeline_all_vars.copy()
    vars_that_stay = ['spot_id', 'start', 'stop', 'event']+\
    [x for x in cols if x not in base_cols]+\
    [x for x in df_timeline.columns if 'spot_category_' in x or 'metro_area_' in x]
    df_timeline.drop([x for x in df_timeline.columns if x not in vars_that_stay], axis = 1, inplace = True)

    ### variables to skip ###
    df_timeline.isnull().sum().sum() #OK
    skip_vars = list((df_timeline!=0).sum()[(df_timeline!=0).sum()==0].index)
    skip_vars

    ctv = fit_tv_cox_models.fit_the_models_and_print_summaries(df_timeline=df_timeline, base_df=base_df, \
                                                         date_of_analysis=date_of_analysis, model_type=model_type, \
                                                         variables_to_use_for_the_model=cols_to_use,\
                                                         coefficient_limit_for_numerical_vars=coefficient_limit_for_numerical_vars, \
                                                         coefficient_limit_for_cat_vars=coefficient_limit_for_cat_vars, p_limit=p_limit, \
                                                         additional_higher_p_limit=additional_higher_p_limit, \
                                                         additional_lower_p_limit=additional_lower_p_limit,\
                                                         skip_vars=skip_vars, penalizer=penalizer)

    ### save coefs and p values for p < 0.2 ###
    fit_tv_cox_models.save_results(df = ctv.summary.reset_index(), date_of_analysis = date_of_analysis, \
                                   data_dir = churn_based_on_behaviour_dir, dir_name='exports/coefficients_and_pvalues',\
                 results_name = 'coef_and_pvalues', spots_set = spots_set,\
                 with_wo_CB = with_wo_CB, event_date_type = event_date_full_name, p_limit=0.2, model_number=model_number,\
                                  model_type = model_type)

# Model 1: posts_on_facebook

## without variables with p >= 0.2

### sorted by variable name

<lifelines.CoxTimeVaryingFitter: fitted with 34817 periods, 2748 subjects, 424 events>
         event col = 'event'
         penalizer = 0.001
number of subjects = 2748
 number of periods = 34817
  number of events = 424
partial log-likelihood = -2733.68
  time fit was run = 2021-12-21 10:07:55 UTC

---
                                                     coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                                  
Monthly                                              0.81       2.25       0.17             0.48             1.14                 1.61                 3.14
New.email.subscribers.allinclusive.last.month.d...  -0.04       0.96       0.02            -0.09             0.00                 0.91                 1.00
Website.Views.last.month.total.log2                 -0.16       0.85       0.04            -0.23            -0.09                 0.79                 0.91
had_added_events_manually_edited_events_before       0.12       1.12       0.13            -0.14             0.37                 0.87                 1.45
had_added_events_manually_edited_events_last_3_...  -0.34       0.71       0.22            -0.77             0.09                 0.46                 1.09
had_catering_submissions_before                     -0.10       0.90       0.17            -0.44             0.24                 0.64                 1.27
had_catering_submissions_last_3_months              -0.47       0.63       0.22            -0.91            -0.03                 0.40                 0.97
had_clicked_emails_before                            0.33       1.39       0.13             0.07             0.58                 1.07                 1.79
had_clicked_emails_last_3_months                    -0.56       0.57       0.13            -0.82            -0.30                 0.44                 0.74
had_emails_sent_manually_scheduled_emails_sent_...   0.06       1.06       0.12            -0.18             0.30                 0.84                 1.35
had_emails_sent_manually_scheduled_emails_sent_...  -0.25       0.78       0.14            -0.52             0.03                 0.59                 1.03
had_other_non_contactless_menu_qr_flyer_scans_b...  -0.28       0.76       0.28            -0.83             0.27                 0.44                 1.31
had_other_non_contactless_menu_qr_flyer_scans_l...  -0.87       0.42       0.50            -1.84             0.11                 0.16                 1.11
had_posts_on_facebook_before                         0.38       1.46       0.13             0.13             0.62                 1.14                 1.87
had_posts_on_facebook_last_month                    -0.49       0.61       0.12            -0.73            -0.25                 0.48                 0.78
had_reservations_submissions_before                  0.22       1.24       0.18            -0.14             0.57                 0.87                 1.78
had_reservations_submissions_last_month             -0.43       0.65       0.28            -0.97             0.11                 0.38                 1.12
had_tickets_before                                  -0.53       0.59       0.12            -0.77            -0.28                 0.46                 0.75
had_tickets_last_3_months                            0.07       1.07       0.13            -0.18             0.32                 0.84                 1.37
had_visited_admin_before                             0.22       1.24       0.21            -0.19             0.63                 0.82                 1.88
had_visited_admin_last_month                        -0.40       0.67       0.11            -0.63            -0.18                 0.53                 0.83
had_visited_regular_flyers_page_before               0.17       1.19       0.12            -0.07           

# Model 2: changed_picture_or_text

...

<lifelines.CoxTimeVaryingFitter: fitted with 34817 periods, 2748 subjects, 424 events>
         event col = 'event'
         penalizer = 0.001
number of subjects = 2748
 number of periods = 34817
  number of events = 424
partial log-likelihood = -2724.83
  time fit was run = 2021-12-21 11:29:15 UTC

---
                                                     coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                                  
Fb.page.likes.analysis.avg.log2                     -0.03       0.97       0.02            -0.06             0.00                 0.94                 1.00
Monthly                                              0.85       2.33       0.17             0.51             1.18                 1.67                 3.26
Website.Views.last.month.total.log2                 -0.17       0.84       0.03            -0.24            -0.11                 0.79                 0.90
had_added_events_manually_edited_events_before       0.18       1.19       0.13            -0.08             0.43                 0.92                 1.54
had_added_events_manually_edited_events_last_3_...  -0.29       0.75       0.22            -0.72             0.14                 0.49                 1.15
had_catering_submissions_before                     -0.13       0.87       0.17            -0.48             0.21                 0.62                 1.23
had_catering_submissions_last_3_months              -0.48       0.62       0.22            -0.91            -0.04                 0.40                 0.96
had_clicked_emails_before                            0.28       1.32       0.13             0.02             0.54                 1.02                 1.72
had_clicked_emails_last_3_months                    -0.52       0.60       0.13            -0.78            -0.26                 0.46                 0.77
had_emails_sent_manually_scheduled_emails_sent_...   0.04       1.04       0.12            -0.20             0.28                 0.82                 1.32
had_emails_sent_manually_scheduled_emails_sent_...  -0.21       0.81       0.14            -0.49             0.07                 0.62                 1.07
had_other_non_contactless_menu_qr_flyer_scans_b...  -0.10       0.91       0.30            -0.68             0.49                 0.51                 1.63
had_other_non_contactless_menu_qr_flyer_scans_l...  -0.85       0.43       0.50            -1.83             0.13                 0.16                 1.14
had_posts_liked_before                               0.02       1.02       0.12            -0.22             0.27                 0.80                 1.30
had_posts_liked_last_3_months                       -0.33       0.72       0.18            -0.67             0.02                 0.51                 1.02
had_posts_on_facebook_before                         0.30       1.35       0.14             0.03             0.57                 1.03                 1.77
had_posts_on_facebook_last_month                    -0.49       0.62       0.12            -0.72            -0.25                 0.48                 0.78
had_posts_seen_before                                0.30       1.35       0.19            -0.06             0.67                 0.94                 1.95
had_posts_seen_last_3_months                         0.03       1.03       0.17            -0.31             0.38                 0.73                 1.46
had_preview_page_views_before                        0.36       1.43       0.20            -0.04             0.75                 0.96                 2.13
had_preview_page_views_last_3_months                -0.09       0.91       0.17            -0.42             0.24                 0.66                 1.27
had_reservations_submissions_before                  0.20       1.22       0.18            -0.16           

# Model 33: New.email.subscribers.allinclusive.last.month.div10

## without variables with p >= 0.2

### sorted by variable name

<lifelines.CoxTimeVaryingFitter: fitted with 34817 periods, 2748 subjects, 424 events>
         event col = 'event'
         penalizer = 0.001
number of subjects = 2748
 number of periods = 34817
  number of events = 424
partial log-likelihood = -2723.09
  time fit was run = 2021-12-21 11:29:45 UTC

---
                                                     coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                                  
Fb.page.likes.analysis.avg.log2                     -0.03       0.97       0.02            -0.06             0.00                 0.94                 1.00
Monthly                                              0.85       2.33       0.17             0.51             1.18                 1.67                 3.26
New.email.subscribers.allinclusive.last.month.d...  -0.04       0.96       0.02            -0.09             0.00                 0.92                 1.00
Website.Views.last.month.total.log2                 -0.16       0.86       0.04            -0.22            -0.09                 0.80                 0.92
had_added_events_manually_edited_events_before       0.17       1.19       0.13            -0.08             0.43                 0.92                 1.54
had_added_events_manually_edited_events_last_3_...  -0.28       0.75       0.22            -0.72             0.15                 0.49                 1.16
had_catering_submissions_before                     -0.13       0.88       0.17            -0.47             0.21                 0.62                 1.23
had_catering_submissions_last_3_months              -0.46       0.63       0.22            -0.89            -0.02                 0.41                 0.98
had_clicked_emails_before                            0.28       1.33       0.13             0.02             0.54                 1.02                 1.72
had_clicked_emails_last_3_months                    -0.52       0.60       0.13            -0.78            -0.25                 0.46                 0.78
had_emails_sent_manually_scheduled_emails_sent_...   0.04       1.04       0.12            -0.20             0.28                 0.82                 1.33
had_emails_sent_manually_scheduled_emails_sent_...  -0.21       0.81       0.14            -0.49             0.07                 0.61                 1.07
had_other_non_contactless_menu_qr_flyer_scans_b...  -0.08       0.93       0.30            -0.66             0.51                 0.52                 1.66
had_other_non_contactless_menu_qr_flyer_scans_l...  -0.84       0.43       0.50            -1.83             0.14                 0.16                 1.15
had_posts_liked_before                               0.02       1.02       0.12            -0.23             0.26                 0.80                 1.29
had_posts_liked_last_3_months                       -0.33       0.72       0.18            -0.67             0.02                 0.51                 1.02
had_posts_on_facebook_before                         0.30       1.35       0.14             0.03             0.57                 1.03                 1.77
had_posts_on_facebook_last_month                    -0.49       0.61       0.12            -0.73            -0.25                 0.48                 0.78
had_posts_seen_before                                0.30       1.35       0.19            -0.06             0.67                 0.94                 1.95
had_posts_seen_last_3_months                         0.03       1.03       0.17            -0.31             0.37                 0.73                 1.45
had_preview_page_views_before                        0.36       1.43       0.20            -0.04             0.76                 0.97                 2.13
had_preview_page_views_last_3_months                -0.08       0.92       0.17            -0.42           